In [1]:
## Install Dependencies
!pip install xgboost gradio joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
from google.colab import files
uploaded = files.upload()

Saving xgboost_model.pkl to xgboost_model.pkl


In [3]:
import os
print("Model exists:", os.path.exists("xgboost_model.pkl"))

Model exists: True


In [4]:
import xgboost as xgb
import joblib
import gradio as gr
import numpy as np

# Load the trained XGBoost model
model = joblib.load("xgboost_model.pkl")

# Define the prediction function
def predict_model(age, gender, income, marital_status, dependents, education, occupation, health, location, policy, claims, vehicle, credit_score, insurance_duration, feedback, smoking, exercise, property):
    print("Received input age:", age)  # Debugging step
    print("Received input gender:", gender)  # Debugging step
    print("Received input income:", income)  # Debugging step

    # Convert categorical inputs to numerical values
    gender_map = {"Male": 0, "Female": 1}
    marital_map = {"Single": 0, "Married": 1, "Divorced": 2}
    education_map = {"High School": 0, "Bachelor's": 1, "Master's": 2, "PhD": 3}
    occupation_map = {"Unemployed": 0, "Self-Employed": 1, "Salaried": 2}
    location_map = {"Urban": [1, 0, 0], "Suburban": [0, 1, 0], "Rural": [0, 0, 1]}
    policy_map = {"Basic": 0, "Premium": 1, "Comprehensive": 2}
    feedback_map = {"Good": 3, "Average": 2, "Poor": 1}
    smoking_map = {"Yes": 1, "No": 0}
    exercise_map = {"Rarely": 0, "Monthly": 1, "Weekly": 2, "Daily": 3}
    property_map = {"House": 0, "Apartment": 1, "Condo": 2}

    # Convert inputs to numeric format
    gender = gender_map[gender]
    marital_status = marital_map[marital_status]
    education = education_map[education]
    occupation = occupation_map[occupation]
    location_values = location_map[location]  # This expands into three separate binary columns
    policy = policy_map[policy]
    feedback = feedback_map[feedback]
    smoking = smoking_map[smoking]
    exercise = exercise_map[exercise]
    property = property_map[property]

    # Compute missing transformed features
    income_credit_score = income / credit_score
    age_claims = age / claims if claims != 0 else 0
    log_annual_income = np.log(income) if income > 0 else 0
    income_per_dependent = income / dependents if dependents > 0 else income
    health_credit = health / credit_score
    log_previous_claims = np.log(claims) if claims > 0 else 0

    # Combine all features in correct order
    features = np.array([[
        claims, income, health, feedback, credit_score, occupation, vehicle,
        insurance_duration, exercise, policy, age, dependents, marital_status,
        education, *location_values, income_credit_score, age_claims,
        log_annual_income, income_per_dependent, health_credit, log_previous_claims
    ]])

    print("Features shape:", features.shape)  # Debugging step
    print("Features array:", features)  # Debugging step

    # Make prediction
    prediction = model.predict(features)[0]
    print("Prediction:", prediction)  # Debugging step
    return prediction

# Create the Gradio web app
iface = gr.Interface(
    fn=predict_model,
    inputs=[
        gr.Number(label="Age"),
        gr.Radio(["Male", "Female"], label="Gender"),
        gr.Number(label="Annual Income"),
        gr.Radio(["Single", "Married", "Divorced"], label="Marital Status"),
        gr.Number(label="Number of Dependents"),
        gr.Radio(["High School", "Bachelor's", "Master's", "PhD"], label="Education Level"),
        gr.Radio(["Unemployed", "Self-Employed", "Salaried"], label="Occupation"),
        gr.Number(label="Health Score"),
        gr.Radio(["Urban", "Suburban", "Rural"], label="Location"),
        gr.Radio(["Basic", "Premium", "Comprehensive"], label="Policy Type"),
        gr.Number(label="Previous Claims"),
        gr.Number(label="Vehicle Age"),
        gr.Number(label="Credit Score"),
        gr.Number(label="Insurance Duration"),
        gr.Radio(["Good", "Average", "Poor"], label="Customer Feedback"),
        gr.Radio(["Yes", "No"], label="Smoking Status"),
        gr.Radio(["Rarely", "Monthly", "Weekly", "Daily"], label="Exercise Frequency"),
        gr.Radio(["House", "Apartment", "Condo"], label="Property Type")
    ],
    outputs="number",
    title="XGBoost Regression Predictor",
    description="Enter details and get predictions from your trained XGBoost model."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1445771221975aba88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
iface.close()  # Shut down any running instance
iface.launch(share=True)  # Relaunch Gradio


Closing server running on port: 7860
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1445771221975aba88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
